In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as Func
import torch.optim as optim
import torch.utils.data as Data
# from gekko import GEKKO
from math import exp
import sys
import os
from time import sleep, time
from tqdm import tqdm
from scipy.optimize import minimize
from collections import OrderedDict
import multiprocessing
from functools import reduce


e = 2.71828182846

T0 = 300.0
V = 1.0
k0 = 8.46e6
Cp = 0.231
sigma = 1000.0
# Ts = 400.0
Ts = 430.0
Qs = 0.0
F = 5.0
E = 5e4
delta_H = -1.15e4
R = 8.314
CAs = 2.0
CA0s = 4.0

P = np.array([[716.83, 0.0], [0.0, 1.0]])
gamma = 9.53    # equation 45g in "economic..."
x_size = 2
u_size = 2
c_hl_size = 8 # hidden layer for critic
a_hl_size = 8 # hidden layer for actor
u1_bd = 3.5
# u2_bd = 5e5
u2_bd = 5.0   # u2 is multified by 1e-5
u2_scale = 1e5
x1_bd = 1.0
x2_bd = 26.0
QMat = np.array([[3.0, 0.0], [0.0, 1.0]])
RMat = np.array([[1.0/20.0, 0.0], [0.0, 1.0/30.0]])

NUM_OF_X1 = 2
NUM_OF_X2 = 3

DISCOUNT = 0.99  # discount for q-learning rewards
C_LR = 1e-2     # learning rate for critic
A_LR = 1e-2     # learning rate for actor
C_MAX_EPOCH = 50
A_MAX_EPOCH = 50
MAX_Q_STEP = 2
BATCH_SIZE = 4

exception_num_0 = 0 # lyapunov constraints may conflict with u's bound
exception_num_1 = 0 # even not considering lyapunov constraints, solution still not found

'''
x=0, u=0 -> dx/dt=0
'''
def getCA0s():
    return V/F * k0 * np.exp(-E/(R*Ts)) * (CAs**2) + CAs

def getQs():
    return sigma * Cp * V * (F/V * (Ts-T0) + delta_H/(sigma*Cp) * k0 * np.exp(-E/(R*Ts)) * (CAs**2))

CA0s = getCA0s()
Qs = getQs()


In [2]:

class GActionNoise():
    def __init__(self, mu, sigma, decay_factor):
        self.mu = np.array(mu)
        self.sigma = np.array(sigma)
        self.decay_factor = decay_factor
    
    def __call__(self):
        x = np.random.normal(self.mu, self.sigma, self.mu.shape)
        return x
    
    def decay(self):
        self.sigma *= self.decay_factor


class ReplayBuffer():
    def __init__(self, max_size, input_size, action_size, max_reward):
        self.mem_size = max_size
        self.mem_cntr = 0
        self.max_reward = max_reward
        
        self.state_memory = np.zeros((self.mem_size, input_size))
        self.action_memory = np.zeros((self.mem_size, action_size))
        self.reward_memory = np.zeros(self.mem_size)
        self.new_state_memory = np.zeros((self.mem_size, input_size))
        self.terminal_memory = np.zeros(self.mem_size, dtype=np.int32)
    
    def store_transition(self, state, action, reward, new_state, done):
        idx = self.mem_cntr % self.mem_size
        
        self.state_memory[idx] = state
        self.action_memory[idx] = action
        self.reward_memory[idx] = reward
        self.new_state_memory[idx] = new_state
        self.terminal_memory[idx] = done
        
        self.mem_cntr += 1
    
    def sample_buffer(self, batch_size):
        max_mem = min(self.mem_cntr, self.mem_size)
        batch = np.random.choice(max_mem, batch_size)

        states = self.state_memory[batch]
        actions = self.action_memory[batch]
        rewards = self.reward_memory[batch]
        new_states = self.new_state_memory[batch]
        terminal = self.terminal_memory[batch]

        return states, actions, rewards, new_states, terminal


class CriticFc(nn.Module):
    def __init__(self, c_lr, input_size, fc1_dims, fc2_dims, action_size, 
                 input_bds, action_bds, name, chkpt_dir='tmp/ddpg'):
        super(CriticFc, self).__init__()
        self.input_size = input_size
        self.action_size = action_size
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.input_bds = np.array(input_bds)
        self.action_bds = np.array(action_bds)
        self.checkpoint_file = os.path.join(chkpt_dir, name+'_ddpg.pkl')

        # normalization layer
        self.nl = nn.Linear(self.input_size + self.action_size, self.input_size + self.action_size)
        state_action_bds = 1.0 / np.concatenate((self.input_bds, self.action_bds))
        state_action_bds = np.diag(state_action_bds)
        self.nl.weight = nn.Parameter(torch.tensor(state_action_bds))
        self.nl.bias = nn.Parameter(torch.zeros(self.input_size + self.action_size))
        for p in self.nl.parameters():
            p.requires_grad = False

        self.fc1 = nn.Linear(self.input_size + self.action_size, self.fc1_dims)
        self.fc2 = nn.Linear(self.fc1_dims, self.fc2_dims)
        self.q = nn.Linear(self.fc2_dims, 1)
        f3 = 0.01
        nn.init.uniform_(self.q.weight.data, -f3, f3)
        nn.init.uniform_(self.q.bias.data, -f3, f3)

        self.optimizer = optim.Adam(filter(lambda p: p.requires_grad, self.parameters()), lr=c_lr)
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.to(self.device)
    
    def forward(self, state, action):
        state_action = torch.cat((state, action), 1)
        state_action = self.nl(state_action)
        state_action_value = torch.relu(self.fc1(state_action))
        state_action_value = torch.relu(self.fc2(state_action_value))
        state_action_value = self.q(state_action_value)

        return state_action_value
    
    def save_checkpoint(self):
        # print("... saving checkpoint ...")
        torch.save(self.state_dict(), self.checkpoint_file)
    
    def load_checkpoint(self):
        print("... loading checkpoint ...")
        self.load_state_dict(torch.load(self.checkpoint_file))


class ActorFc(nn.Module):
    def __init__(self, a_lr, input_size, fc1_dims, fc2_dims, action_size,
                 input_bds, action_bds, name, chkpt_dir='tmp/ddpg'):
        super(ActorFc, self).__init__()
        self.input_size = input_size
        self.action_size = action_size
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.input_bds = np.array(input_bds)
        self.action_bds = np.array(action_bds)
        self.checkpoint_file = os.path.join(chkpt_dir, name + '_ddpg.pkl')

        # normalization layer
        self.nl = nn.Linear(self.input_size, self.input_size)
        state_bds = np.diag(1.0 / self.input_bds)
        self.nl.weight = nn.Parameter(torch.tensor(state_bds))
        self.nl.bias = nn.Parameter(torch.zeros(self.input_size))
        for p in self.nl.parameters():
            p.requires_grad = False

        self.fc1 = nn.Linear(self.input_size, self.fc1_dims)
        self.fc2 = nn.Linear(self.fc1_dims, self.fc2_dims)
        self.u = nn.Linear(self.fc2_dims, self.action_size)
        f3 = 0.01
        nn.init.uniform_(self.u.weight.data, -f3, f3)
        nn.init.uniform_(self.u.bias.data, -f3, f3)

        # scaling layer
        self.sl = nn.Linear(self.action_size, self.action_size)
        action_bds = np.diag(self.action_bds)
        self.sl.weight = nn.Parameter(torch.tensor(action_bds))
        self.sl.bias = nn.Parameter(torch.zeros(self.action_size))
        for p in self.sl.parameters():
            p.requires_grad = False

        self.optimizer = optim.Adam(filter(lambda p: p.requires_grad, self.parameters()), lr=a_lr)
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.to(self.device)
    
    def forward(self, state):
        state = self.nl(state)
        action = torch.relu(self.fc1(state))
        action = torch.relu(self.fc2(action))
        action = torch.tanh(self.u(action))
        action = self.sl(action)

        return action

    def save_checkpoint(self):
        # print("... saving checkpoint ...")
        torch.save(self.state_dict(), self.checkpoint_file)
    
    def load_checkpoint(self):
        print("... loading checkpoint ...")
        self.load_state_dict(torch.load(self.checkpoint_file))


class AgentFc():
    def __init__(self, a_lr, c_lr, input_size, action_size, input_bds, action_bds, max_reward,
                 action_noise_mu, action_noise_sigma, action_noise_decay, tau,
                 env, discount=0.99, max_size=10000, layer1_size=12,
                 layer2_size=8, batch_size=64, chkpt_dir='tmp/ddpg'):
        self.discount = discount
        self.tau = tau
        self.memory = ReplayBuffer(max_size, input_size, action_size, max_reward)
        self.batch_size = batch_size

        self.critic = CriticFc(c_lr, input_size, layer1_size, layer2_size,
                               action_size, input_bds, action_bds, 'Critic', chkpt_dir).double()
        self.actor = ActorFc(a_lr, input_size, layer1_size, layer2_size,
                             action_size, input_bds, action_bds, 'Actor', chkpt_dir).double()
        self.target_critic = CriticFc(c_lr, input_size, layer1_size, layer2_size,
                                      action_size, input_bds, action_bds, 'TargetCritic', chkpt_dir).double()
        self.target_actor = ActorFc(a_lr, input_size, layer1_size, layer2_size,
                                    action_size, input_bds, action_bds, 'TargetActor', chkpt_dir).double()
        
        self.noise = GActionNoise(action_noise_mu, action_noise_sigma, action_noise_decay)

        self.update_network_parameters(tau=1)
    
    def choose_action(self, observation, add_noise):
        observation = torch.tensor(observation).double().to(self.actor.device)
        u = self.actor(observation).to(self.actor.device)
        if add_noise:
            u += torch.tensor(self.noise()).double().to(self.actor.device)
        
        return u.cpu().detach().numpy()
    
    def remember(self, state, action, reward, new_state, done):
        self.memory.store_transition(state, action, reward, new_state, done)

    def get_target_critic_value(self, reward, new_critic_value_tensor, done):
        '''
        'new_critic_value_tensor' is a tensor containing a single element,
        so use new_critic_value_tensor.item() to get its value

        if done == 0:
            not a terminal state
        if done == 1:
            exceed the range
        if done == 2:
            converge to 0
        '''
        ### argmax ###
        # if done == 0:
        #     return reward + self.discount * new_critic_value_tensor.item()
        # if done == 1:
        #     return reward + 0.0
        # return reward + self.memory.max_reward

        ### argmin ###
        if done == 0:
            return reward + self.discount * new_critic_value_tensor.item()
        if done == 2:
            return reward + 0.0
        return reward + self.memory.max_reward


    def learn(self):
        if self.memory.mem_cntr < self.batch_size:
            return
        state, action, reward, new_state, done = self.memory.sample_buffer(self.batch_size)
        
        state = torch.tensor(state).double().to(self.critic.device)
        action = torch.tensor(action).double().to(self.critic.device)
        reward = torch.tensor(reward).double().to(self.critic.device)
        new_state = torch.tensor(new_state).double().to(self.critic.device)

        new_action = self.target_actor(new_state)
        new_critic_value = self.target_critic(new_state, new_action)
        critic_value = self.critic(state, action)

        target_critic_value = list(map(self.get_target_critic_value, reward, new_critic_value, done))
        target_critic_value = torch.tensor(target_critic_value).to(self.critic.device)
        target_critic_value = target_critic_value.view(self.batch_size, 1)

        self.critic.optimizer.zero_grad()
        loss = nn.MSELoss()
        critic_loss = loss(target_critic_value, critic_value)
        critic_loss.backward()
        self.critic.optimizer.step()

        u = self.actor(state)
        self.actor.optimizer.zero_grad()
        # actor_loss = -self.critic(state, u) # argmax
        actor_loss = self.critic(state, u)  # argmin
        actor_loss = torch.mean(actor_loss)
        actor_loss.backward()
        self.actor.optimizer.step()
        
        self.update_network_parameters()
    
    def update_network_parameters(self, tau=None):
        if tau is None:
            tau = self.tau

        actor_params = self.actor.named_parameters()
        critic_params = self.critic.named_parameters()
        target_actor_params = self.target_actor.named_parameters()
        target_critic_params = self.target_critic.named_parameters()

        critic_dict = dict(critic_params)
        actor_dict = dict(actor_params)
        target_critic_dict = dict(target_critic_params)
        target_actor_dict = dict(target_actor_params)

        for name in critic_dict:
            critic_dict[name] = tau * critic_dict[name].clone() + \
                                (1-tau) * target_critic_dict[name].clone()

        self.target_critic.load_state_dict(critic_dict)
        
        for name in actor_dict:
            actor_dict[name] = tau * actor_dict[name].clone() + \
                                (1-tau) * target_actor_dict[name].clone()
        
        self.target_actor.load_state_dict(actor_dict)
    
    def save_models(self):
        self.actor.save_checkpoint()
        self.critic.save_checkpoint()
        self.target_actor.save_checkpoint()
        self.target_critic.save_checkpoint()
    
    def load_models(self):
        self.actor.load_checkpoint()
        self.critic.load_checkpoint()
        self.target_actor.load_checkpoint()
        self.target_critic.load_checkpoint()


In [3]:
class CstrEnv():
    def __init__(self, obs_flag, he=1e-4, hs=0.01, x0=np.zeros(2)):
        self.x = x0.copy()
        self.he = he
        self.hs = hs
        self.obs_flag = obs_flag
    
    def get_reward(self, xk, uk):
        # reward_x = 1.0 / (np.fabs(xk[0]) + 1.0) + 1.0 / (np.fabs(xk[1]) + 1.0)
        # # if self.obs_flag == '1':
        # #     reward_x = 1.0 / (np.fabs(xk[0]) + 1.0)
        # # elif self.obs_falg == '2':
        # #     reward_x = 1.0 / (np.fabs(xk[1]) + 1.0)

        # reward_u = 0.01 / (np.fabs(uk[0]) + 1.0) + 0.01 / (np.fabs(uk[1]) + 1.0)

        # return reward_x + reward_u

        reward_x = 3 * np.sqrt(np.fabs(xk[0])) + np.sqrt(np.fabs(xk[1]))
        reward_u = (uk**2 / np.array([20.0, 30.0])).sum()

        return reward_x + reward_u

    
    def get_new_state(self, xk, uk, add_noise=False, noise=None):
        xk1 = xk.copy()
        if not add_noise:
            for _ in range(int(self.hs / self.he)):
                xk1[0] += self.he * (F/V * (uk[0] + CA0s - xk1[0] - CAs) - k0 * np.exp(-E/R/(xk1[1] + Ts)) * (xk1[0] + CAs)**2)
                xk1[1] += self.he * (F/V * (T0 - xk1[1] - Ts) + (-delta_H)/sigma/Cp * k0 * np.exp(-E/R/(xk1[1] + Ts)) * (xk1[0] + CAs)**2 + (uk[1] * u2_scale  + Qs)/sigma/Cp/V)
        else:
            for _ in range(int(self.hs / self.he)):
                xk1[0] += self.he * (F/V * (uk[0] + CA0s - xk1[0] - CAs) - k0 * np.exp(-E/R/(xk1[1] + Ts)) * (xk1[0] + CAs)**2) + self.he * noise[0]
                xk1[1] += self.he * (F/V * (T0 - xk1[1] - Ts) + (-delta_H)/sigma/Cp * k0 * np.exp(-E/R/(xk1[1] + Ts)) * (xk1[0] + CAs)**2 + (uk[1] * u2_scale  + Qs)/sigma/Cp/V) + self.he * noise[1]

        return xk1
    
    def reset(self, x0=None):
        if x0 is None:
            self.x = np.random.uniform([-x1_bd, -x2_bd], [x1_bd, x2_bd], size=(2,))
        else:
            self.x = x0.copy()
        
        if self.obs_flag == '1':
            return np.array([self.x[0]])
        elif self.obs_flag == '2':
            return np.array([self.x[1]])
        return self.x
    
    def step(self, uk, add_noise=False, noise=None):
        reward = self.get_reward(self.x, uk)
        new_state = self.get_new_state(self.x, uk, add_noise, noise)
        done = 0
        if abs(new_state[0]) > x1_bd or abs(new_state[1]) > x2_bd:
            done = 1
        elif abs(new_state[0]) < 0.001 and abs(new_state[1]) < 0.001:
            done = 2
        
        self.x = new_state.copy()

        if self.obs_flag == '1':
            new_state = np.array([new_state[0]])
        elif self.obs_flag == '2':
            new_state = np.array([new_state[1]])
        
        return new_state, reward, done

In [4]:
env = CstrEnv('12', x0=np.random.uniform([-x1_bd, -x2_bd], [x1_bd, x2_bd], size=(2,)))
agent = AgentFc(a_lr=1e-4, c_lr=1e-3, input_size=2, action_size=2,
                input_bds=[x1_bd, x2_bd], action_bds=[u1_bd, u2_bd], max_reward=500,
                action_noise_mu=[0.0, 0.0], action_noise_sigma=[0.3, 0.005],
                action_noise_decay=0.5, tau=0.005, env=env, discount=0.98,
                max_size=100000, layer1_size=400, layer2_size=300,
                batch_size=64, chkpt_dir='/kaggle/working')

# agent.load_models()

np.random.seed(0)

rewards = []
steps = []

rewards_file = '/kaggle/working/rewards.npy'
steps_file = '/kaggle/working/steps.npy'

In [5]:
# agent.load_models()


for episode in tqdm(range(1, 5001)):
    obs = env.reset()
    done = 0
    step = 0
    total_reward = 0
    
    while not done and step < 200:
        step += 1
        act = agent.choose_action(obs, add_noise=True)
        new_state, reward, done = env.step(act)
        agent.remember(obs, act, reward, new_state, done)
        agent.learn()
        obs = new_state
        
        total_reward += reward
    
    # print(done)
    if done == 1:
        total_reward += 500
    
    rewards.append(total_reward)
    steps.append(step)
    
    if episode % 200 == 0:
        agent.save_models()
        np.save(rewards_file, np.array(rewards))
        np.save(steps_file, np.array(steps))
    
    # if episode % 1000 == 0:
    #     agent.noise.decay()

agent.save_models()

rewards = np.array(rewards)
steps = np.array(steps)
np.save(rewards_file, rewards)
np.save(steps_file, steps)

100%|██████████| 5000/5000 [5:43:17<00:00,  4.12s/it]


In [6]:
# r = np.load('/kaggle/working/rewards.npy')
# s = np.load('/kaggle/working/steps.npy')
# print(len(r), len(s))
# plt.figure()
# plt.plot(range(len(r)), r)
# plt.figure()
# plt.plot(range(len(s)), s)
# plt.figure()
# plt.show()